# 05: Real-World Architecture — Enforcing a Design System

This notebook applies KindScript to a **real production codebase**: the Induction Studio storybook, a React design system with ~300 source files following Atomic Design principles.

We'll progressively add architectural enforcement:

**Part 1 — Explore:** See the real codebase structure and the implicit rules

**Part 2 — First enforcement:** Add top-level layer constraints, discover real violations

**Part 3 — Iterate:** Refine the model, then test `filesystem.exists`

**Part 4 — Page internals:** Enforce internal structure within a page component (multi-instance)

**Part 5 — The gap:** Demonstrate what KindScript can't yet enforce (sibling isolation)

**Prerequisites:**
- Run `npx tsc` from the project root to compile the CLI
- SSH access to `git@github.com:inductionAI/studio.git` (private repo)

## Setup

Clones the real storybook codebase at a pinned git SHA for reproducibility. First run takes ~5 seconds (shallow clone); subsequent runs reuse the cached clone in `targets/`.

In [ ]:
import { PROJECT_ROOT, ksc, tree, loadStorybook, writeFile, readFile, cleanup } from './lib.ts';

const DEMO = await loadStorybook();
console.log("Working directory:", DEMO);

In [ ]:
// How big is this codebase?
const countCmd = new Deno.Command("find", {
  args: [DEMO + "/src", "-type", "f", "(", "-name", "*.ts", "-o", "-name", "*.tsx", ")",
         "-not", "-path", "*/node_modules/*"],
  stdout: "piped",
});
const { stdout: countOut } = await countCmd.output();
const fileCount = new TextDecoder().decode(countOut).trim().split("\n").length;
console.log(`${fileCount} TypeScript/TSX files in the storybook`);

---

## Part 1: The Architecture (Before KindScript)

This is a real design system following **Atomic Design** — a hierarchy of component complexity:

```
atoms       → pure presentational (Button, Input, Icon)
molecules   → composite UI (Card, Alert, Form, Sidebar)
organisms   → domain containers with business logic (ReleasesManager, DocumentManager)
pages       → full features (DashboardPage, SettingsPage, ReleasesPage)
```

The rules exist in documentation and code review:
- Atoms can't depend on molecules, organisms, or pages
- Molecules can't depend on organisms or pages
- Organisms can't depend on pages
- Production code shouldn't depend on mock data
- Core infrastructure (auth, providers) is independent of UI layers

But nothing *enforces* these rules. Let's look at the real code.

In [ ]:
// Show the directory structure (top 4 levels)
await tree(DEMO + "/src");

### A real atom (Button)

Atoms are pure presentational — they depend only on styling utilities. Look at the imports.

In [ ]:
const buttonSrc = readFile(DEMO, "src/components/atoms/Button/v1.0.0/Button.tsx");
console.log("=== Button.tsx (first 15 lines) ===");
console.log(buttonSrc.split("\n").slice(0, 15).join("\n"));

### A real molecule (Card)

Molecules compose atoms. Card imports only from `common/` — no organisms or pages.

In [ ]:
const cardSrc = readFile(DEMO, "src/components/molecules/Card/v1.0.0/Card.tsx");
console.log("=== Card.tsx (first 15 lines) ===");
console.log(cardSrc.split("\n").slice(0, 15).join("\n"));

### A real organism (ReleasesManager)

Organisms are self-contained feature units with colocated business logic, services, and types.

In [ ]:
const rmSrc = readFile(DEMO, "src/components/organisms/ReleasesManager/v1.0.0/ReleasesManager.tsx");
console.log("=== ReleasesManager.tsx (first 15 lines) ===");
console.log(rmSrc.split("\n").slice(0, 15).join("\n"));

---

## Part 2: First Enforcement — The Layer Hierarchy

Let's write our first definition file. We define the atomic design layers as KindScript members and use `noDependency` constraints to enforce the hierarchy.

This is our first pass — we'll learn from the results and refine.

In [ ]:
writeFile(DEMO, "src/context.ts", `
import type { Kind, InstanceConfig } from 'kindscript';

type AtomLayer = Kind<"AtomLayer">;
type MoleculeLayer = Kind<"MoleculeLayer">;
type OrganismLayer = Kind<"OrganismLayer">;
type PageLayer = Kind<"PageLayer">;
type CoreLayer = Kind<"CoreLayer">;
type CommonLayer = Kind<"CommonLayer">;
type MockLayer = Kind<"MockLayer">;

type DesignSystem = Kind<"DesignSystem", {
  atoms: AtomLayer;
  molecules: MoleculeLayer;
  organisms: OrganismLayer;
  pages: PageLayer;
  core: CoreLayer;
  common: CommonLayer;
  mocks: MockLayer;
}, {
  noDependency: [
    // Atoms: leaf nodes
    ["atoms", "molecules"], ["atoms", "organisms"], ["atoms", "pages"],
    // Molecules: can use atoms, nothing above
    ["molecules", "organisms"], ["molecules", "pages"],
    // Organisms: can't depend on pages
    ["organisms", "pages"],
    // Core: independent of UI layers
    ["core", "atoms"], ["core", "molecules"], ["core", "organisms"], ["core", "pages"],
    // Production code can't depend on mocks
    ["atoms", "mocks"], ["molecules", "mocks"], ["organisms", "mocks"],
    ["pages", "mocks"], ["core", "mocks"],
  ];
  filesystem: {
    exists: ["atoms", "molecules", "organisms", "pages", "core"];
  };
}>;

export const designSystem = {
  atoms: {},
  molecules: {},
  organisms: {},
  pages: {},
  core: {},
  common: {},
  mocks: {},
} satisfies InstanceConfig<DesignSystem>;
`);

console.log("Wrote src/context.ts — first-pass model (7 members, 16 noDependency pairs)");

In [ ]:
// Let's see what happens on the real codebase!
const v1 = await ksc("check", DEMO);

// Parse the violations so we can see what's going on
if (v1.code !== 0) {
  const lines = v1.output.split("\n").filter(l => l.includes("error KS"));
  console.log(`\n--- ${lines.length} violation(s) found. Violating files: ---`);
  const files = [...new Set(lines.map(l => {
    const rel = l.replace(DEMO + "/src/", "").split(":")[0];
    return rel;
  }))];
  files.forEach(f => console.log("  " + f));
}

### What we learned from the first pass

The violations fall into two categories:

**1. ServiceProvider crosses layers** — `core/providers/ServiceProvider.tsx` is the dependency injection composition root. It *legitimately* imports services from organisms and pages to wire them together. Our model needs to account for this.

**2. Test/story files import mocks** — Files like `DashboardPage.test.tsx` and `*.stories.tsx` import MSW handlers from `mocks/`. Tests need mock data — this is expected.

These aren't bugs in the codebase. They're places where our *model* doesn't match reality yet. This is the normal adoption process: run, learn, refine.

---

## Part 3: Refine the Model

Two changes:

1. **ServiceProvider**: Split `core` into `providers` (no constraints — it's the composition root) and `auth` (keeps its UI-independence constraints).

2. **Test/story → mock imports**: Remove the `pages -> mocks` constraint. Test files colocated in page directories legitimately need mock data.

In [ ]:
writeFile(DEMO, "src/context.ts", `
import type { Kind, InstanceConfig } from 'kindscript';

type AtomLayer = Kind<"AtomLayer">;
type MoleculeLayer = Kind<"MoleculeLayer">;
type OrganismLayer = Kind<"OrganismLayer">;
type PageLayer = Kind<"PageLayer">;
type ProvidersLayer = Kind<"ProvidersLayer">;   // DI composition root
type AuthLayer = Kind<"AuthLayer">;             // Auth infrastructure
type CommonLayer = Kind<"CommonLayer">;
type MockLayer = Kind<"MockLayer">;

type DesignSystem = Kind<"DesignSystem", {
  atoms: AtomLayer;
  molecules: MoleculeLayer;
  organisms: OrganismLayer;
  pages: PageLayer;
  providers: ProvidersLayer;
  auth: AuthLayer;
  common: CommonLayer;
  mocks: MockLayer;
}, {
  noDependency: [
    ["atoms", "molecules"], ["atoms", "organisms"], ["atoms", "pages"],
    ["molecules", "organisms"], ["molecules", "pages"],
    ["organisms", "pages"],
    ["auth", "atoms"], ["auth", "molecules"], ["auth", "organisms"], ["auth", "pages"],
    ["atoms", "mocks"], ["molecules", "mocks"], ["organisms", "mocks"],
  ];
  filesystem: {
    exists: ["atoms", "molecules", "organisms", "pages"];
  };
}>;

export const designSystem = {
  atoms: {},
  molecules: {},
  organisms: {},
  pages: {},
  providers: {},
  auth: {},
  common: {},
  mocks: {},
} satisfies InstanceConfig<DesignSystem>;
`);

console.log("Wrote refined src/context.ts (8 members, 13 noDependency + 4 exists)");

In [ ]:
await ksc("check", DEMO);

All contracts satisfied. The refined model matches reality:

- **`providers`** has no constraints — it's the DI composition root, allowed to import from anywhere
- **`auth`** is separated from providers and has its own UI-independence constraints
- **`pages -> mocks`** constraint removed — test files in pages legitimately need mock data
- The core architectural hierarchy is still enforced

### Now let's break something

What happens if someone adds a forbidden import? Let's have an atom import from an organism.

In [ ]:
// Save the clean version
const cleanButton = readFile(DEMO, "src/components/atoms/Button/v1.0.0/Button.tsx");

// Inject a forbidden import: atom -> organism
writeFile(DEMO, "src/components/atoms/Button/v1.0.0/Button.tsx",
  `import { ReleaseService } from '../../../organisms/ReleasesManager/v1.0.0/release.service';\n` +
  cleanButton
);

console.log("Injected violation: Button.tsx now imports ReleaseService from organisms");

In [ ]:
await ksc("check", DEMO);

In [ ]:
// Restore the clean version and verify
writeFile(DEMO, "src/components/atoms/Button/v1.0.0/Button.tsx", cleanButton);
await ksc("check", DEMO);

### Testing `filesystem.exists`

Our model declares `filesystem: { exists: ["atoms", "molecules", "organisms", "pages"] }`. This means KindScript verifies these directories actually exist on disk. What happens if one goes missing?

In [ ]:
// Rename molecules → molecules_backup to simulate a missing directory
Deno.renameSync(DEMO + "/src/components/molecules", DEMO + "/src/components/molecules_backup");
console.log("Renamed molecules → molecules_backup");

await ksc("check", DEMO);

In [ ]:
// Restore
Deno.renameSync(DEMO + "/src/components/molecules_backup", DEMO + "/src/components/molecules");
console.log("Restored molecules directory");
await ksc("check", DEMO);

---

## Part 4: Page Internal Architecture

Pages in this codebase have their own internal layered architecture. DashboardPage is the most mature example.

In [ ]:
// Show the real DashboardPage internal structure
const dashBase = "/src/components/Pages/DashboardPage/v1.0.0";
const dashCmd = new Deno.Command("find", {
  args: [DEMO + dashBase, "-type", "d", "-not", "-path", "*/node_modules/*"],
  stdout: "piped",
});
const { stdout: dashOut } = await dashCmd.output();
const dashDirs = new TextDecoder().decode(dashOut).trim().split("\n")
  .map(d => d.replace(DEMO + dashBase + "/", "")).sort();
console.log("DashboardPage/v1.0.0/ directories:");
console.log(dashDirs.join("\n"));

The dependency flow should be: `ui -> domain -> data -> types`. Let's enforce this with a **second definition file** scoped to the DashboardPage.

The root is inferred from the definition file's directory — so placing it inside `v1.0.0/` scopes it to just that page.

In [ ]:
writeFile(DEMO, "src/components/Pages/DashboardPage/v1.0.0/dashboard.ts", `
import type { Kind, InstanceConfig } from 'kindscript';

type UILayer = Kind<"UILayer">;
type DomainLayer = Kind<"DomainLayer">;
type DataLayer = Kind<"DataLayer">;
type TypesLayer = Kind<"TypesLayer">;
type ValidationLayer = Kind<"ValidationLayer">;

type PageArchitecture = Kind<"PageArchitecture", {
  ui: UILayer;
  domain: DomainLayer;
  data: DataLayer;
  types: TypesLayer;
  validation: ValidationLayer;
}, {
  noDependency: [
    ["types", "ui"], ["types", "domain"], ["types", "data"],
    ["validation", "ui"], ["validation", "domain"],
    ["data", "ui"],
    ["domain", "ui"],
  ];
}>;

export const page = {
  ui: {},
  domain: {},
  data: {},
  types: {},
  validation: {},
} satisfies InstanceConfig<PageArchitecture>;
`);

console.log("Wrote dashboard.ts — page-level architecture");
console.log("Two definition files now coexist: top-level (src/) + page-level (DashboardPage/v1.0.0/)");

In [ ]:
// Check both levels at once — KindScript discovers all definition files
await ksc("check", DEMO);

### Let's break a page-internal rule

The `types` layer shouldn't import from `ui`. Let's inject that violation.

In [ ]:
// Find a real types file in DashboardPage
const typesDir = DEMO + "/src/components/Pages/DashboardPage/v1.0.0/types";
const typesFiles: string[] = [];
for (const entry of Deno.readDirSync(typesDir)) {
  if (entry.isFile && entry.name.endsWith(".ts")) typesFiles.push(entry.name);
}
const typesFile = typesFiles[0];
console.log("Target file:", typesFile);

const cleanTypes = readFile(DEMO, `src/components/Pages/DashboardPage/v1.0.0/types/${typesFile}`);

// Inject: types imports from ui (forbidden)
writeFile(DEMO, `src/components/Pages/DashboardPage/v1.0.0/types/${typesFile}`,
  `import { DashboardPage } from '../ui/DashboardPage';\n` + cleanTypes
);

console.log(`Injected violation: types/${typesFile} now imports from ui/`);

In [ ]:
await ksc("check", DEMO);

In [ ]:
// Restore and verify
writeFile(DEMO, `src/components/Pages/DashboardPage/v1.0.0/types/${typesFile}`, cleanTypes);
await ksc("check", DEMO);

---

## Part 5: The Gap — Sibling Isolation

The most important architectural rule in a design system is: **pages don't import from other pages** and **organisms don't import from other organisms**. Each is an independent vertical slice.

Can KindScript enforce this today?

In [ ]:
// Inject a cross-organism import: ReleasesManager imports from DocumentManager
const cleanRM = readFile(DEMO, "src/components/organisms/ReleasesManager/v1.0.0/ReleasesManager.tsx");

writeFile(DEMO, "src/components/organisms/ReleasesManager/v1.0.0/ReleasesManager.tsx",
  `import { DocumentService } from '../../DocumentManager/v1.0.0/document.service';\n` +
  cleanRM
);

console.log("Injected cross-organism import: ReleasesManager -> DocumentManager");

In [ ]:
await ksc("check", DEMO);

**It passes.** KindScript doesn't catch the cross-organism import.

Why? Because `organisms` is a single member. KindScript enforces boundaries *between* members, not *within* a member. It can't see that `ReleasesManager/` and `DocumentManager/` are siblings that shouldn't talk to each other.

### The workaround (verbose)

You could declare every organism as a separate member — but that's O(n²) `noDependency` pairs. For 12 organisms that's 132 pairs. Not practical.

### The solution (proposed: `isolated` constraint)

A single constraint that means "no child of this member can import from any other child":

```typescript
type OrganismLayer = Kind<"OrganismLayer", {}, { isolated: true }>;
type PageLayer = Kind<"PageLayer", {}, { isolated: true }>;
```

One line replaces O(n²) pairs. This is the highest-value feature for real-world design system enforcement.

In [ ]:
// Restore organism and do final clean check (both definition files still active)
writeFile(DEMO, "src/components/organisms/ReleasesManager/v1.0.0/ReleasesManager.tsx", cleanRM);
await ksc("check", DEMO);

---

## Summary

We enforced architecture on a **real production codebase** (~300 files) with ~30 lines of definition code.

### What we learned

| Step | What happened |
|------|---------------|
| First pass | Found real violations — ServiceProvider crosses layers, tests import mocks |
| Refined model | Gave the DI container its own member, relaxed test->mock constraint |
| Injected violation | Atom importing from organism — caught immediately |
| Directory missing | Renamed molecules — `filesystem.exists` caught it |
| Page internals | Added per-page definition file — multi-instance enforces internal layers |
| Page violation | types importing from ui — caught at page level |
| Sibling isolation | Cross-organism import — NOT caught (motivates `isolated` feature) |

### Enforcement matrix

| Rule | Enforced? | How |
|------|-----------|-----|
| Atoms can't import organisms/pages | **Yes** | `noDependency` |
| Molecules can't import pages | **Yes** | `noDependency` |
| Organisms can't import pages | **Yes** | `noDependency` |
| Production code can't import mocks | **Yes** | `noDependency` |
| Page internal layers (ui->domain->data) | **Yes** | Multi-instance definition file |
| Key directories must exist | **Yes** | `filesystem.exists` (demonstrated) |
| Pages can't import each other | **No** | Needs `isolated` |
| Organisms can't import each other | **No** | Needs `isolated` |
| Atoms can't import zustand/tanstack | **No** | Needs `deniedExternals` |

### Key insight: incremental adoption

KindScript doesn't require perfect modeling from day one. The first-pass violations weren't bugs — they revealed where our model didn't match reality. Run, learn, refine, repeat.

In [ ]:
cleanup(DEMO);
console.log("Working directory cleaned up.");